# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 53 new papers today
          12 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/11 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2303.00787


extracting tarball to tmp_2303.00787...

 done.
Retrieving document from  https://arxiv.org/e-print/2303.00898


extracting tarball to tmp_2303.00898...

...

 done.
Retrieving document from  https://arxiv.org/e-print/2303.01356


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2303.01203/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:312: LatexWarning: Latex injecting: 'affiliations.tex' from 'tmp_2303.01203/affiliations.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:312: LatexWarning: Latex injecting: 'authors.tex' from 'tmp_2303.01203/authors.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


extracting tarball to tmp_2303.01356...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2303.01356/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:312: LatexWarning: Latex injecting: './sections/conclusions.tex' from 'tmp_2303.01356/sections/conclusions.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:312: LatexWarning: Latex injecting: './sections/validation.tex' from 'tmp_2303.01356/sections/validation.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Pyth

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.00787-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.00787) | **A flash of polarized optical light points to an aspherical "cow"**  |
|| J.R. Maund, et al. -- incl., <mark>R. J. Smith</mark> |
|*Appeared on*| *2023-03-03*|
|*Comments*| *MNRAS Accepted, 10 pages, 8 figures*|
|**Abstract**| The astronomical transient AT2018cow is the closest example of the new class of luminous, fast blue optical transients (FBOTs). Liverpool Telescope RINGO3 observations of AT2018cow are reported here, which constitute the earliest polarimetric observations of an FBOT. At 5.7 days post-explosion, the optical emission of AT2018cow exhibited a chromatic polarization spike that reached ~7% at red wavelengths. This is the highest intrinsic polarization recorded for a non-relativistic explosive transient, and is observed in multiple bands and at multiple epochs over the first night of observations, before rapidly declining. The apparent wavelength dependence of the polarization may arise through depolarization or dilution of the polarized flux, due to conditions in AT~2018cow at early times. A second ``bump" in the polarization is observed at blue wavelengths at ~12 days. Such a high polarization requires an extremely aspherical geometry that is only apparent for a brief period (<1 day), such as shock breakout through an optically thick disk. For a disk-like configuration, the ratio of the thickness to radial extent must be ~10%. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.00898-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.00898) | **Synchrotron Radiation Dominates the Extremely Bright GRB 221009A**  |
|| Jun Yang, et al. -- incl., <mark>Xiao-Hong Zhao</mark>, <mark>Yan-Qiu Zhang</mark>, <mark>Xin-Qiao Li</mark>, <mark>Zihan Li</mark>, <mark>Lang Shao</mark>, <mark>Li-Ming Song</mark>, <mark>Bing Zhang</mark>, <mark>Fan Zhang</mark>, <mark>Shuai Zhang</mark>, <mark>Shuang-Nan Zhang</mark>, <mark>Bin-Bin Zhang</mark> |
|*Appeared on*| *2023-03-03*|
|*Comments*| *11 pages, 6 figures, 2 tables*|
|**Abstract**| The brightest Gamma-ray burst, GRB 221009A, has spurred numerous theoretical investigations, with particular attention paid to the origins of ultra-high energy TeV photons during the prompt phase. However, analyzing the mechanism of radiation of photons in the $\sim$MeV range has been difficult because the high flux causes pile-up and saturation effects in most GRB detectors. In this letter, we present systematic modeling of the time-resolved spectra of the GRB using unsaturated data obtained from Fermi/GBM (precursor) and SATech-01/GECAM-C (main emission and flare). Our approach incorporates the synchrotron radiation model, which assumes an expanding emission region with relativistic speed and a global magnetic field that decays with radius, and successfully fits such a model to the observational data. Our results indicate that the spectra of the burst are fully in accordance with a synchrotron origin from relativistic electrons accelerated at a large emission radius. The lack of thermal emission in the prompt emission spectra supports a Poynting-flux-dominated jet composition. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.01014-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.01014) | **Spectral analysis of a parsec-scale jet in M87: Observational constraint  on the magnetic field strengths in the jet**  |
|| Hyunwook Ro, et al. -- incl., <mark>Jee-Won Lee</mark>, <mark>Guang-Yao Zhao</mark>, <mark>Jeong Ae Lee</mark>, <mark>Yingkang Zhang</mark>, <mark>Lang Cui</mark>, <mark>Sang-Sung Lee</mark> |
|*Appeared on*| *2023-03-03*|
|*Comments*| *16 pages, 9 figures. Accepted for publication in Astronomy & Astrophysics*|
|**Abstract**| Because of its proximity and the large size of its black hole, M87 is one of the best targets for studying the launching mechanism of active galactic nucleus jets. Currently, magnetic fields are considered to be an essential factor in the launching and accelerating of the jet. However, current observational estimates of the magnetic field strength of the M87 jet are limited to the innermost part of the jet or to HST-1. No attempt has yet been made to measure the magnetic field strength in between. We aim to infer the magnetic field strength of the M87 jet out to a distance of several thousand $r_s$ by tracking the distance-dependent changes in the synchrotron spectrum of the jet from high-resolution very long baseline interferometry observations. In order to obtain high-quality spectral index maps, quasi-simultaneous observations at 22 and 43 GHz were conducted using the KVN and VERA Array (KaVA) and the VLBA. We compared the spectral index distributions obtained from the observations with a model and placed limits on the magnetic field strengths as a function of distance. The overall spectral morphology is broadly consistent over the course of these observations. The observed synchrotron spectrum rapidly steepens from $\alpha_{22-43 GHz}$ ~ -0.7 at ~ 2 mas to $\alpha_{22-43 GHz}$ ~ -2.5 at ~ 6 mas. A spectral index model in which nonthermal electron injections inside the jet decrease with distance can adequately reproduce the observed trend. This suggests the magnetic field strength of the jet at a distance of 2 - 10 mas (~ 900 $r_s$ - ~ 4500 $r_s$ in the deprojected distance) has a range of $B=(0.3 - 1.0 G)(z/2 mas)^{-0.73}$. Extrapolating to the EHT scale yields consistent results, suggesting that the majority of the magnetic flux of the jet near the black hole is preserved out to ~ 4500 $r_s$ without significant dissipation. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.01174-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.01174) | **Astronomical puzzle Cyg X-3 is a hidden Galactic ultraluminous X-ray  source**  |
|| Alexandra Veledina, et al. -- incl., <mark>David A. Green</mark> |
|*Appeared on*| *2023-03-03*|
|*Comments*| *52 pages, 18 figures, 4 tables*|
|**Abstract**| How black holes consume and eject matter has been the subject of intense studies for more than 60 years. The luminosity of these systems are often compared to the Eddington limit, the border at which the spherical accretion is inhibited by the radiation pressure of photons it produces. The discovery of ultraluminous X-ray sources (ULXs) showed that accretion can proceed even when the apparent luminosity exceeds the Eddington limit (Kaaret et al. 2017). High apparent luminosity might be produced by the beaming of the incident radiation by a thick collimated outflow or by a truly super-Eddington accretion flow. However, possibilities to study these outflows in detail are limited, as ULXs are typically found in distant galaxies. Using the Imaging X-ray Polarimetry Explorer (IXPE, Weisskopf et al. 2022), we made the first measurement of X-ray polarization in Galactic X-ray binary Cyg X-3. The detection of high, $\approx$25\%, nearly energy-independent linear polarization, orthogonal to the direction of the radio ejections, unambiguously indicates the primary source is obscured and the observer on Earth only sees reflected and scattered light. Modelling shows there is an optically thick envelope with a narrow funnel around the primary X-ray source in the system. We derive an upper limit on the opening angle of the funnel that implies a lower limit on the beamed luminosity exceeding the Eddington value. We show that Cyg X-3 is viewed as a ULX to an extragalactic observer located along the axis of the funnel. Our findings reveal this unique persistent source as an ideal laboratory for the study of the inner workings of ULX central engines. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.01198-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.01198) | **Growth-rate measurement with type-Ia supernovae using ZTF survey  simulations**  |
|| Bastien Carreres, et al. -- incl., <mark>Mathew Smith</mark>, <mark>Frank J. Masci</mark> |
|*Appeared on*| *2023-03-03*|
|*Comments*| *20 pages, 13 figures*|
|**Abstract**| Measurements of the growth rate of structures at $z < 0.1$ with peculiar velocity surveys have the potential of testing the validity of general relativity on cosmic scales. In this work, we present growth-rate measurements from realistic simulated sets of type-Ia supernovae (SNe Ia) from the Zwicky Transient Facility (ZTF). We describe our simulation methodology, the light-curve fitting and peculiar velocity estimation. Using the maximum likelihood method, we derive constraints on $f\sigma_8$ using only ZTF SN Ia peculiar velocities. We carefully tested the method and we quantified biases due to selection effects (photometric detection, spectroscopic follow-up for typing) on several independent realizations. We simulated the equivalent of 6 years of ZTF data, and considering an unbiased spectroscopically typed sample at $z < 0.06$, we obtained unbiased estimates of $f\sigma_8$ with an average uncertainty of 19% precision. We also investigated the information gain in applying bias correction methods. Our results validate our framework which can be used on real ZTF data. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.01203-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.01203) | **Insight-HXMT and GECAM-C observations of the brightest-of-all-time GRB  221009A**  |
|| Zheng-Hua An, et al. -- incl., <mark>Li Chen</mark>, <mark>He Gao</mark>, <mark>Dong-Ya Guo</mark>, <mark>Zhi-Wei Guo</mark>, <mark>Jian-Jian He</mark>, <mark>Jun-Wang He</mark>, <mark>An Li</mark>, <mark>Bing Li</mark>, <mark>Chao-Yang Li</mark>, <mark>Cheng-Kui Li</mark>, <mark>Gang Li</mark>, <mark>Mao-Shun Li</mark>, <mark>Ti-Pei Li</mark>, <mark>Wei Li</mark>, <mark>Xiao-Bo Li</mark>, <mark>Xin-Qiao Li</mark>, <mark>Xu-Fang Li</mark>, <mark>Yan-Guo Li</mark>, <mark>Zheng-Wei Li</mark> |
|*Appeared on*| *2023-03-03*|
|*Comments*| *Submitted to National Science Review. This paper is under press embargo, contact the corresponding author for details*|
|**Abstract**| GRB 221009A is the brightest gamma-ray burst ever detected since the discovery of this kind of energetic explosions. However, an accurate measurement of the prompt emission properties of this burst is very challenging due to its exceptional brightness. With joint observations of \textit{Insight}-HXMT and GECAM-C, we made an unprecedentedly accurate measurement of the emission during the first $\sim$1800 s of GRB 221009A, including its precursor, main emission (ME, which dominates the burst in flux), flaring emission and early afterglow, in the hard X-ray to soft gamma-ray band from $\sim$ 10 keV to $\sim$ 6 MeV. Based on the GECAM-C unsaturated data of the ME, we measure a record-breaking isotropic equivalent energy ($E_{\rm iso}$) of $\bf \sim 1.5 \times 10^{55}$ erg, which is about eight times the total rest-mass energy of the Sun. The early afterglow data require a significant jet break between 650 s and 1100 s, most likely at $\sim950$ s from the afterglow starting time $T_{AG}$, which corresponds to a jet opening angle of $\sim {0.7^\circ} \ (\eta_\gamma n)^{1/8}$, where $n$ is the ambient medium density in units of $\rm cm^{-3}$ and $\eta_\gamma$ is the ratio between $\gamma$-ray energy and afterglow kinetic energy. The beaming-corrected total $\gamma$-ray energy $E_{\gamma}$ is $\sim 1.15 \times10^{51} \ (\eta_\gamma n)^{1/4}$ erg, which is typical for long GRBs. These results suggest that this GRB may have a special central engine, which could launch and collimate a very narrowly beamed jet with an ordinary energy budget, leading to exceptionally luminous gamma-ray radiation per unit solid angle. Alternatively, more GRBs might have such a narrow and bright beam, which are missed by an unfavorable viewing angle or have been detected without distance measurement. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.01356-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.01356) | **An Implementation of a Channelizer based on a Goertzel Filter Bank for  the Read-Out of Cryogenic Sensors**  |
|| L. P. Ferreyro, et al. -- incl., <mark>N. Müller</mark> |
|*Appeared on*| *2023-03-03*|
|*Comments*| *17 pages, 18 figures, paper submitted for review for the Journal of Instrumentation (JINST)*|
|**Abstract**| In this work we present an application of the Goertzel Filter for the channelization of multi-tonal signals, typically used for the read-out of cryogenic sensors which are multiplexed in the frequency domain (FDM), by means of Microwave Superconducting Quantum Interference Device (SQUID) Multiplexer ($\mu$MUX). We demonstrate how implementing a bank of many of these filters, can be used to perform a channelization of the multi-tonal input signal to retrieve the data added by the sensors. We show how this approach can be implemented in a resource-efficient manner in a Field Programmable Gate Array (FPGA) within the state-of-the-art, which allows great scalability for reading thousands of sensors; as is required by Radio Telescopes in Cosmic Microwave Background Radiation (CMB) surveys using cryogenic bolometers, particles detection like Neutrino mass estimation using cryogenic calorimeters or Quantum Computing. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.00961-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.00961) | **Observational constraints on non-cold dark matter and phenomenological  emergent dark energy**  |
|| Yan-Hong Yao, Jun-Chao Wang, <mark>Xin-He Meng</mark> |
|*Appeared on*| *2023-03-03*|
|*Comments*| *7 pages, 2 figure*|
|**Abstract**| Although almost all of the models proposed to resolve several long-standing problems making $\Lambda$CDM imperfect assume that dark matter is pressureless, the probability that dark matter is non-cold is still not excluded by current observations. Therefore, in this article, we treat the equation of state for dark matter as a free parameter, and apply observational data to investigate the non-coldness of dark matter. Impressing by the simplicity of the phenomenological emergent dark energy (PEDE) and its ability to relieve the Hubble tension, we propose the PEDE+$w$DM model based on PEDE and non-cold dark matter. We then place constraints on this model in light of the Planck 2018 Cosmic Microwave Background (CMB) anisotropies, baryon acoustic oscillation(BAO) measurements and the Pantheon compilation of Type Ia supernovae. The results indicate a preference for a negative dark matter equation of state at $95\%$ CL for all data sets except CMB alone, which suggests that the non-coldness assumption of dark matter worth to be investigated further in order to understand the nature of dark matter. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'utf-8' codec can't decode byte 0xa1 in position 697: invalid start byte</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

217  publications in the last 7 days.
	 _build/html/2303.00044.md
	 _build/html/2303.00012.md
	 _build/html/2302.14137.md
	 _build/html/2302.12805.md
	 _build/html/2302.10943.md
	 _build/html/2302.10528.md
	 _build/html/2302.10008.md
	 _build/html/2302.08962.md
	 _build/html/2302.08628.md
	 _build/html/2302.07916.md
	 _build/html/2302.07880.md
	 _build/html/2302.07497.md
	 _build/html/2302.07277.md
	 _build/html/2302.07256.md
	 _build/html/2302.07234.md
	 _build/html/2302.07057.md
	 _build/html/2302.05694.md
	 _build/html/2302.05465.md
	 _build/html/2302.04507.md
	 _build/html/2302.04239.md
	 _build/html/2302.04138.md
	 _build/html/2302.03699.md
	 _build/html/2302.03576.md
	 _build/html/2302.03042.md
	 _build/html/2302.03039.md
	 _build/html/2302.02890.md
	 _build/html/2302.02611.md
	 _build/html/2302.02429.md
	 _build/html/2302.01678.md
	 _build/html/2302.00450.md
	 _build/html/2301.13766.md
	 _build/html/2301.13526.md
	 _build/html/2301.12890.md
	 _build/html/2301.11942.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> daily </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.
